In [35]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# JSON Agent

This notebook showcases an agent designed to interact with large JSON/dict objects. This is useful when you want to answer questions about a JSON blob that's too large to fit in the context window of an LLM. The agent is able to iteratively explore the blob to find what it needs to answer the user's question.

In the below example, we are using the OpenAPI spec for the OpenAI API, which you can find [here](https://github.com/openai/openai-openapi/blob/master/openapi.yaml).

We will use the JSON agent to answer some questions about the API spec.

## Initialization

In [56]:
import os
import yaml

from langchain.agents import (
    create_json_agent,
    AgentExecutor
)
from langchain.agents.agent_toolkits import JsonToolkit
from langchain.chains import LLMChain
from langchain.llms.openai import OpenAI
from langchain.requests import TextRequestsWrapper
from langchain.tools.json.tool import JsonSpec

from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI

In [57]:
# agent_executor = create_python_agent(
#     llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
#     tool=tool,
#     verbose=True,
#     agent_type=AgentType.OPENAI_FUNCTIONS
# )

In [58]:
# data['paths'].keys()

In [59]:
# with open("openai_openapi.yml") as f:
#     data = yaml.load(f, Loader=yaml.FullLoader)

In [74]:
data = {
    'contacts':
    {
      "name": "John Doe",
      "age": 30,
      "email": "johndoe@example.com",
      "address": 
        {
            "main": {
            "street": "123 Main St",
            "city": "New York",
            "state": "NY",
            "zipcode": "10001",
            "phone numbers":
                [
                    '123456',
                    '654321'
                ]
          },
            "secondary": {
            "street": "456 Main St",
            "city": "New York",
            "state": "NY",
            "zipcode": "10001"  
            },
        },
      "phone_numbers": [
        {
          "type": "home",
          "number": "555-1234"
        },
        {
          "type": "work",
          "number": "555-5678"
        }
      ],
      "friends": [
        {
          "name": "Jane Smith",
          "age": 28,
          "email": "janesmith@example.com"
        },
        {
          "name": "Tom Johnson",
          "age": 32,
          "email": "tomjohnson@example.com"
        }
      ],
    }
}

In [75]:
json_spec = JsonSpec(dict_=data, max_value_length=4000)
json_toolkit = JsonToolkit(spec=json_spec)

In [76]:
# json_toolkit.get_tools()[0]("data['paths']")

## Example: getting the required POST parameters for a request

In [77]:
json_agent_executor = create_json_agent(
    llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
    toolkit=json_toolkit,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS
)

In [78]:
json_agent_executor.run("What are John Doe's main address and what are his phone numbers?")

Error in on_chain_start callback: 'name'



Invoking: `json_spec_list_keys` with `data["contacts"]`


['name', 'age', 'email', 'address', 'phone_numbers', 'friends']
Invoking: `json_spec_get_value` with `data["contacts"]["address"][0]`


KeyError(0)
Invoking: `json_spec_get_value` with `data["contacts"]["address"]`


{'main': {'street': '123 Main St', 'city': 'New York', 'state': 'NY', 'zipcode': '10001', 'phone numbers': ['123456', '654321']}, 'secondary': {'street': '456 Main St', 'city': 'New York', 'state': 'NY', 'zipcode': '10001'}}John Doe's main address is 123 Main St, New York, NY, 10001. His phone numbers are 123456 and 654321.

> Finished chain.


"John Doe's main address is 123 Main St, New York, NY, 10001. His phone numbers are 123456 and 654321."

In [47]:
json_agent_executor.run("What are the required parameters in the request body to the /completions endpoint?")

Error in on_chain_start callback: 'name'


Prompt [SystemMessage(content='You are an agent designed to interact with JSON.\nYour goal is to return a final answer by interacting with the JSON.\nYou have access to the following tools which help you learn more about the JSON you are interacting with.\nOnly use the below tools. Only use the information returned by the below tools to construct your final answer.\nDo not make up any information that is not contained in the JSON.\nYour input to the tools should be in the form of `data["key"][0]` where `data` is the JSON blob you are interacting with, and the syntax used is Python.\nBe careful to always use double quotes to surround your keys, as in the example. \nYou should only use keys that you know for a fact exist. You must validate that a key exists by seeing it previously when calling `json_spec_list_keys`. \nIf you have not seen a key in one of those responses, you cannot use it.\nYou should only add one key at a time to the path. You cannot add multiple keys at once.\nIf you e

{'__arg1': 'data["paths"]["/completions"]["post"]["requestBody"]["content"]["application/json"]["schema"]["$ref"]'}
Agent decision _FunctionsAgentAction(tool='json_spec_get_value', tool_input='data["paths"]["/completions"]["post"]["requestBody"]["content"]["application/json"]["schema"]["$ref"]', log='\nInvoking: `json_spec_get_value` with `data["paths"]["/completions"]["post"]["requestBody"]["content"]["application/json"]["schema"]["$ref"]`\n\n\n', message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'json_spec_get_value', 'arguments': '{\n  "__arg1": "data[\\"paths\\"][\\"/completions\\"][\\"post\\"][\\"requestBody\\"][\\"content\\"][\\"application/json\\"][\\"schema\\"][\\"$ref\\"]"\n}'}}, example=False)])

Invoking: `json_spec_get_value` with `data["paths"]["/completions"]["post"]["requestBody"]["content"]["application/json"]["schema"]["$ref"]`


#/components/schemas/CreateCompletionRequestPrompt [SystemMessage(content='You are an agent designed to interac


KeyboardInterrupt



In [14]:
json_agent_executor = create_json_agent(
#     llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
    llm=OpenAI(),
    toolkit=json_toolkit,
    verbose=True)

In [18]:
json_agent_executor.run("What are the required parameters in the request body to the /completions endpoint?")

Error in on_chain_start callback: 'name'


Action: json_spec_list_keys
Action Input: data
Observation: ['openapi', 'info', 'servers', 'tags', 'paths', 'components', 'x-oaiMeta']
Thought: I should look at the "path" key
Action: json_spec_list_keys
Action Input: data["paths"]
Observation: ['/engines', '/engines/{engine_id}', '/completions', '/edits', '/images/generations', '/images/edits', '/images/variations', '/embeddings', '/engines/{engine_id}/search', '/files', '/files/{file_id}', '/files/{file_id}/content', '/answers', '/classifications', '/fine-tunes', '/fine-tunes/{fine_tune_id}', '/fine-tunes/{fine_tune_id}/cancel', '/fine-tunes/{fine_tune_id}/events', '/models', '/models/{model}', '/moderations']
Thought: I should look at the "/completions" key
Action: json_spec_list_keys
Action Input: data["paths"]["/completions"]
Observation: ['post']
Thought:


KeyboardInterrupt

